### Part 2

Hungarian algorithm

In [37]:
import numpy as np

class State:
    def __init__(self, cost_matrix):
        self.cm = cost_matrix.copy()

        n, m = self.cm.shape
        self.row_uncovered = np.ones(n, dtype=bool)
        self.col_uncovered = np.ones(m, dtype=bool)
        self.marked = np.zeros((n, m), dtype=int)   # 1: starred, 2: primed
        self.Z0_r = 0
        self.Z0_c = 0
        self.path = np.zeros((n + m, 2), dtype=int)

    def clean_covers(self):
        self.row_uncovered[:] = True
        self.col_uncovered[:] = True

def step6(state):
    if np.any(state.row_uncovered) and np.any(state.col_uncovered):
        minval = np.min(state.cm[state.row_uncovered], axis=0)
        minval = np.min(minval[state.col_uncovered])
        state.cm[~state.row_uncovered] += minval
        state.cm[:, state.col_uncovered] -= minval
    return step4

def step5(state):
    count = 0
    path = state.path
    path[count, 0] = state.Z0_r
    path[count, 1] = state.Z0_c

    while True:
        row = np.argmax(state.marked[:, path[count, 1]] == 1)
        if state.marked[row, path[count, 1]] != 1:
            break
        else:
            count += 1
            path[count, 0] = row
            path[count, 1] = path[count - 1, 1]

        col = np.argmax(state.marked[path[count, 0]] == 2)
        if state.marked[row, col] != 2:
            col = -1
        count += 1
        path[count, 0] = path[count - 1, 0]
        path[count, 1] = col

    for i in range(count + 1):
        if state.marked[path[i, 0], path[i, 1]] == 1:
            state.marked[path[i, 0], path[i, 1]] = 0
        else:
            state.marked[path[i, 0], path[i, 1]] = 1

    state.clean_covers()
    state.marked[state.marked == 2] = 0
    return step3

def step4(state):
    # Accelerate
    C = (state.cm == 0).astype(int)
    covered_C = C * state.row_uncovered[:, np.newaxis]
    covered_C *= np.asarray(state.col_uncovered, dtype=int)
    
    while True:
        row, col = np.unravel_index(np.argmax(covered_C), (state.cm.shape[0], state.cm.shape[1]))   # find 0s
        if covered_C[row, col] == 0:
            return step6
        else:
            state.marked[row, col] = 2
            star_col = np.argmax(state.marked[row] == 1)
            if state.marked[row, star_col] != 1:
                state.Z0_r = row
                state.Z0_c = col
                return step5
            else:
                col = star_col
                state.row_uncovered[row] = False    # change the assignment
                state.col_uncovered[col] = True
                covered_C[:, col] = C[:, col] * (np.asarray(state.row_uncovered, dtype=int))
                covered_C[row] = 0

def step3(state):
    starred = (state.marked == 1)
    state.col_uncovered[np.any(starred, axis=0)] = False 

    if starred.sum() < state.cm.shape[0]:
        return step4

# Based on https://brc2.com/the-algorithm-workshop/
def my_hungarian(cost_matrix):
    cost_matrix = np.asarray(cost_matrix)
    # Step 0
    flag = False
    if cost_matrix.shape[1] < cost_matrix.shape[0]:
        cost_matrix = cost_matrix.T
        flag = True

    state = State(cost_matrix)

    # Step 1
    state.cm -= state.cm.min(axis=1)[:, np.newaxis]

    # Step 2
    for i, j in zip(*np.where(state.cm == 0)):
        if state.col_uncovered[j] and state.row_uncovered[i]:
            state.marked[i, j] = 1
            state.col_uncovered[j] = False
            state.row_uncovered[i] = False
    
    state.clean_covers()
    
    step = step3
    while step is not None:
        step = step(state)

    marked = state.marked
    if flag:
        marked = state.marked.T
    return np.where(marked == 1)


In [38]:
import json
import cv2 as cv
import numpy as np
# from scipy.optimize import linear_sum_assignment

def load_obj_each_frame(data_file):
    with open(data_file, 'r') as file:
        frame_dict = json.load(file)
    return frame_dict

class AlphaBetaFilter:
    def __init__(self, alpha, beta, dt):
        self.alpha = alpha
        self.beta = beta
        self.dt = dt

    def predict(self, position, velocity):
        predicted_position = position + velocity * self.dt
        return predicted_position

    def update(self, predicted_position, current_measurement):
        residual = current_measurement - predicted_position
        position = predicted_position + self.alpha * residual
        velocity = (self.beta * residual) / self.dt
        return position, velocity

# Calculate the centroid of a bounding box
def calculate_centroid(x_min, y_min, width, height):
    return np.array([x_min + width / 2, y_min + height / 2])

# Track and assign IDs to objects
def track_objects(frame_dict):
    tracked_objects = []
    next_id = 0
    default_id = 0

    for frame, detections in frame_dict.items():
        centroids = [calculate_centroid(obj['x_min'], obj['y_min'], obj['width'], obj['height']) for obj in detections]
        centroids_np = np.array(centroids) 
        # print(centroids_np.shape)

        if not tracked_objects:
            # Initialize tracked objects
            for obj in detections:
                obj['id'] = default_id
                default_id += 1
            for centroid in centroids_np:
                tracked_objects.append({
                    'filter': AlphaBetaFilter(alpha=0.85, beta=0.005, dt=1),
                    'position': centroid,
                    'velocity': np.array([0, 0]),
                    'id': next_id
                })
                next_id += 1

        else:
            # Predict positions of tracked objects
            predicted_positions = np.array([obj['filter'].predict(obj['position'], obj['velocity']) for obj in tracked_objects])
            # print(predicted_positions.shape)

            # Calculate distances between predictions and detections
            cost_matrix = np.linalg.norm(predicted_positions[:, np.newaxis] - centroids_np, axis=2)
            print(cost_matrix.shape)
            
            # Find the optimal assignment
            # row_ind, col_ind = linear_sum_assignment(cost_matrix)   # already Hungarian
            row_ind, col_ind = my_hungarian(cost_matrix)
            
            # Update and assign IDs
            used_ids = set()
            for row, col in zip(row_ind, col_ind):
                if cost_matrix[row, col] < 50: 
                    obj = tracked_objects[row]
                    centroid = centroids_np[col]
                    position, velocity = obj['filter'].update(obj['position'], centroid)
                    obj['position'] = position
                    obj['velocity'] = velocity
                    detections[col]['id'] = obj['id']
                    used_ids.add(obj['id'])
            # Assign new IDs
            for i, detection in enumerate(detections):
                if 'id' not in detection: 
                    detection['id'] = next_id
                    centroid = centroids_np[i]
                    tracked_objects.append({
                        'filter': AlphaBetaFilter(alpha=0.85, beta=0.0005, dt=1),
                        'position': centroid,
                        'velocity': np.array([0, 0]),
                        'id': next_id
                    })
                    next_id += 1

    return frame_dict

def draw_object(object_dict, image):
    x = object_dict['x_min']
    y = object_dict['y_min']
    width = object_dict['width']
    height = object_dict['height']
    object_id = object_dict.get('id', 'N/A')
    cv.rectangle(image, (x, y), (x + width, y + height), (0, 255, 0), 2)
    cv.putText(image, f'ID: {object_id}', (x, y-10), cv.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

def draw_objects_in_video(video_file, frame_dict):
    cap = cv.VideoCapture(video_file)
    vidwrite = cv.VideoWriter("part_2_demo.mp4", cv.VideoWriter_fourcc(*'MP4V'), 30, (700, 500))
    frame_count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv.resize(frame, (700, 500))
        if str(frame_count) in frame_dict:
            for obj in frame_dict[str(frame_count)]:
                draw_object(obj, frame)
        vidwrite.write(frame)
        frame_count += 1
    cap.release()
    vidwrite.release()

In [39]:
data_file = 'frame_dict.json'
frame_dict = load_obj_each_frame(data_file)
frame_dict_with_ids = track_objects(frame_dict) 
with open('part_2_frame_dict.json', 'w', encoding='utf-8') as file:
    json.dump(frame_dict_with_ids, file, ensure_ascii=False, indent=None)
print("Successfully saved in part_2_frame_dict.json!")

video_file = "commonwealth.mp4"
draw_objects_in_video(video_file, frame_dict_with_ids)

(6, 5)
(7, 7)
(8, 7)
(8, 7)
(9, 7)
(9, 7)
(10, 6)
(10, 7)
(10, 9)
(12, 7)
(12, 7)
(12, 7)
(12, 6)
(12, 7)
(12, 7)
(12, 8)
(12, 7)
(12, 8)
(12, 9)
(12, 8)
(12, 6)
(12, 6)
(12, 5)
(12, 4)
(12, 4)
(12, 4)
(12, 5)
(13, 7)
(13, 6)
(13, 6)
(13, 8)
(13, 6)
(13, 7)
(13, 7)
(13, 5)
(13, 5)
(13, 6)
(13, 5)
(13, 5)
(13, 6)
(13, 6)
(13, 7)
(13, 4)
(13, 4)
(13, 6)
(13, 6)
(13, 9)
(13, 7)
(13, 7)
(13, 6)
(13, 7)
(13, 7)
(13, 7)
(13, 8)
(13, 5)
(13, 5)
(13, 6)
(13, 7)
(13, 8)
(13, 7)
(13, 8)
(14, 8)
(14, 9)
(14, 8)
(14, 7)
(14, 7)
(14, 6)
(14, 7)
(14, 6)
(14, 6)
(14, 6)
(14, 5)
(14, 5)
(14, 4)
(14, 3)
(14, 5)
(14, 4)
(14, 6)
(15, 5)
(15, 4)
(15, 5)
(15, 6)
(15, 5)
(15, 7)
(15, 6)
(15, 5)
(15, 6)
(15, 4)
(15, 4)
(15, 5)
(15, 3)
(15, 4)
(15, 4)
(15, 4)
(15, 4)
(15, 4)
(15, 4)
(15, 4)
(15, 5)
(15, 5)
(15, 5)
(15, 4)
(15, 5)
(15, 6)
(15, 6)
(15, 7)
(15, 6)
(15, 6)
(15, 8)
(15, 7)
(15, 8)
(15, 7)
(15, 7)
(15, 7)
(15, 7)
(15, 7)
(15, 7)
(15, 7)
(15, 7)
(15, 6)
(15, 7)
(15, 8)
(15, 8)
(15, 8)
(15, 7)
(15, 6

OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
